In [ ]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np

from dynamicslib.consts import muEM
from dynamicslib.continuation import find_bif, arclen_cont
from dynamicslib.common import prop_ic_fullstate, get_JC_tf, eom,f_df_CR3_single, coupled_stm_eom
from dynamicslib.targeter import dc_arclen, dc_square, dc_overconstrained, dc_arclen
from dynamicslib.integrate import dop853
from tqdm.auto import tqdm
from dynamicslib.plotting import plotly_display, to_eci
from dynamicslib.common_targetters import spatial_perpendicular, spatial_period_fixed, fullstate_minus_one

family = "Resonant 1-2"

root = os.getcwd()
file = f"{root}/database/{family}.csv"
db = pd.read_csv(file).set_index("Index")

In [ ]:
targetter = fullstate_minus_one(1, 0, 0.0, 1e-11, muEM)
func = targetter.f_df_stm


def X2xtf(X):
    return targetter.get_x0(X), targetter.get_tf(X)

In [ ]:
keys = [
    "Initial x",
    "Initial y",
    "Initial z",
    "Initial vx",
    "Initial vy",
    "Initial vz",
    "Period",
]
for key in keys:
    if key not in db.columns:
        db[key] = 0.0

Xm0 = np.delete(db[keys].iloc[1110].values, 1)
Xm1 = np.delete(db[keys].iloc[1109].values, 1)

In [ ]:
X0, _ = find_bif(
    Xm0,
    func,
    (Xm0 - Xm1),
    1e-3,
    targ_tol=1e-11,
    bisect_tol=1e-9,
    bif_type="tangent",
    debug=True,
)

In [ ]:
# # # switch direction!
# x0, tf = targetter.get_x0(X0), targetter.get_tf(X0)
# ts, xs, _, _ = dop853(eom, (0, tf / 2), x0, 1e-14, 0)
# xs[1, -1] = 0
# X0 = targetter.get_X(xs[:, -1], ts[-1] * 2)

In [ ]:
X1 = X0.copy()
# X1[-1] *= 3
f, df, stm = func(X1)
svd = np.linalg.svd(df)
tangent = svd.Vh[-2]
tangent

In [ ]:
Xsp1, esp1 = arclen_cont(
    X1, func, tangent, 1e-3, 0.01, 1e-10, exact_tangent=True, max_iter=300
)

In [ ]:
Xsp2, esp2 = arclen_cont(
    Xsp1[-1], func, Xsp1[-1] - Xsp1[-2], 5e-3, 0.1, 1e-10, max_iter=250
)

In [ ]:
Xsm1, esm1 = arclen_cont(
    X1, func, -tangent, 5e-3, 0.5, 1e-9, exact_tangent=True, max_iter=200
)

In [ ]:
Xsm2, esm2 = arclen_cont(
    Xsm1[-1], func, Xsm1[-1] - Xsm1[-2], 5e-3, 1.5, 1e-9, max_iter=250
)

In [ ]:
# f, df, stm = func(Xsp10[-1])
# svd = np.linalg.svd(df)
# tngt = svd.Vh[-2]

In [ ]:
# _, df, _ = func(Xsp2[-1])
# tangent = np.linalg.svd(df).Vh[-1]

# Xsp3, esp3 = arclen_to_fail(Xsp2[-1], func, tangent, 5e-3, 1e-8, 30)

In [ ]:
Xsp = [*Xsp1, *Xsp2[1:]]
esp = [*esp1, *esp2[1:]]
Xsm = [*Xsm1, *Xsm2[1:]]
esm = [*esm1, *esm2[1:]]

param_names = [
    "Index",
    "Initial x",
    "Initial y",
    "Initial z",
    "Initial vx",
    "Initial vy",
    "Initial vz",
    "Period",
    "Jacobi Constant",
    "Stability Index",
    "Eig1",
    "Eig2",
    "Eig3",
    "Eig4",
    "Eig5",
    "Eig6",
]

xyzs = []
data = []


iterator = list(enumerate(zip(Xsm, esm)))[::-1]
for i, (X, evals) in tqdm(iterator):
    Xcp = X.copy()
    xyzs.append(prop_ic_fullstate(Xcp, X2xtf, int_tol=1e-10))
    args = np.argsort(np.abs(evals))
    stabind = np.max([(np.abs(lam) + 1 / np.abs(lam)) / 2 for lam in evals])
    jc, period = get_JC_tf(Xcp, X2xtf)
    x0 = targetter.get_x0(Xcp)
    data.append([-i, *x0, period, jc, stabind, *evals])

iterator = list(enumerate(zip(Xsp, esp)))
for i, (X, evals) in tqdm(iterator):
    Xcp = X.copy()
    # Xcp[-1]
    xyzs.append(prop_ic_fullstate(Xcp, X2xtf, int_tol=1e-10))
    args = np.argsort(np.abs(evals))
    stabind = np.max([(np.abs(lam) + 1 / np.abs(lam)) / 2 for lam in evals])
    jc, period = get_JC_tf(Xcp, X2xtf)
    x0 = targetter.get_x0(Xcp)
    data.append([i, *x0, period, jc, stabind, *evals])

In [ ]:
df = pd.DataFrame(data, columns=param_names).set_index("Index")
for key in keys:
    if np.all(df[key].abs() < 1e-10):
        df = df.drop(columns=key)
        
df.to_csv("New Family.csv")

In [ ]:
a = None
b = None
c = 5
plotly_display(xyzs[a:b:c][::-1], df[a:b:c][::-1])